In [7]:
import pyspark
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

24/02/22 02:56:37 WARN Utils: Your hostname, codespaces-74f203 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/02/22 02:56:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/22 02:56:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

--2024-02-22 02:57:34--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 108.138.245.225, 108.138.245.16, 108.138.245.58, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|108.138.245.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-01.parquet’

fhvhv_tripdata_2021 100%[===================>] 294.61M  20.6MB/s    in 16s     

2024-02-22 02:57:51 (18.9 MB/s) - ‘fhvhv_tripdata_2021-01.parquet’ saved [308924937/308924937]



In [9]:
!wc -l fhvhv_tripdata_2021-01.parquet

1006794 fhvhv_tripdata_2021-01.parquet


In [10]:
df = spark.read \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [16]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_re

In [21]:
df = df.repartition(24)

In [22]:
df.write.parquet('fhvhv/2021/01/')

In [24]:
df = spark.read \
    .parquet('fhvhv/2021/01/')

In [25]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_f

In [29]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .filter(df.hvfhs_license_num == 'HV0003') \
    .head(5)

[Row(pickup_datetime=datetime.datetime(2021, 1, 5, 12, 42, 9), dropoff_datetime=datetime.datetime(2021, 1, 5, 13, 4, 44), PULocationID=22, DOLocationID=261),
 Row(pickup_datetime=datetime.datetime(2021, 1, 2, 14, 16, 6), dropoff_datetime=datetime.datetime(2021, 1, 2, 14, 22, 35), PULocationID=189, DOLocationID=181),
 Row(pickup_datetime=datetime.datetime(2021, 1, 6, 8, 26, 45), dropoff_datetime=datetime.datetime(2021, 1, 6, 8, 53), PULocationID=56, DOLocationID=75),
 Row(pickup_datetime=datetime.datetime(2021, 1, 28, 1, 7, 33), dropoff_datetime=datetime.datetime(2021, 1, 28, 1, 17, 39), PULocationID=169, DOLocationID=69),
 Row(pickup_datetime=datetime.datetime(2021, 1, 17, 9, 0, 11), dropoff_datetime=datetime.datetime(2021, 1, 17, 9, 13, 5), PULocationID=169, DOLocationID=69)]

In [30]:
from pyspark.sql import functions

In [39]:
df \
    .withColumn('pickup_date', functions.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', functions.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .filter(df.hvfhs_license_num == 'HV0003') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-05|  2021-01-05|          22|         261|
| 2021-01-02|  2021-01-02|         189|         181|
| 2021-01-06|  2021-01-06|          56|          75|
| 2021-01-28|  2021-01-28|         169|          69|
| 2021-01-17|  2021-01-17|         169|          69|
| 2021-01-14|  2021-01-14|         164|         145|
| 2021-01-12|  2021-01-12|         169|         250|
| 2021-01-06|  2021-01-06|          74|         152|
| 2021-01-22|  2021-01-22|          37|          80|
| 2021-01-01|  2021-01-01|         248|         213|
| 2021-01-19|  2021-01-19|          91|          14|
| 2021-01-05|  2021-01-05|         236|         255|
| 2021-01-31|  2021-01-31|         229|         145|
| 2021-01-14|  2021-01-14|         135|         265|
| 2021-01-12|  2021-01-12|         210|          89|
| 2021-01-16|  2021-01-16|          26|       

In [40]:
from pyspark.sql import types

In [43]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

crazy_stuff_udf = functions.udf(crazy_stuff, returnType=types.StringType())

In [44]:
df \
    .withColumn('pickup_date', functions.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', functions.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .filter(df.hvfhs_license_num == 'HV0003') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/acc| 2021-01-05|  2021-01-05|          22|         261|
|  e/b33| 2021-01-02|  2021-01-02|         189|         181|
|  s/acd| 2021-01-06|  2021-01-06|          56|          75|
|  s/acd| 2021-01-28|  2021-01-28|         169|          69|
|  e/b3b| 2021-01-17|  2021-01-17|         169|          69|
|  e/acc| 2021-01-14|  2021-01-14|         164|         145|
|  e/95b| 2021-01-12|  2021-01-12|         169|         250|
|  e/b47| 2021-01-06|  2021-01-06|          74|         152|
|  e/b30| 2021-01-22|  2021-01-22|          37|          80|
|  s/b44| 2021-01-01|  2021-01-01|         248|         213|
|  e/b48| 2021-01-19|  2021-01-19|          91|          14|
|  s/acd| 2021-01-05|  2021-01-05|         236|         255|
|  e/acc| 2021-01-31|  2021-01-31|         229|         145|
|  e/acc| 2021-01-14|  2